In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import torch.utils
from torchvision import datasets,transforms

In [6]:
batch_size=200
learning_rate=0.01
epochs=10

train_db = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size=batch_size, shuffle=True)

test_db = datasets.MNIST('../data', train=False, 
    transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))
test_loader = torch.utils.data.DataLoader(
    test_db,
    batch_size=batch_size, shuffle=True)

In [7]:
print('train length: {}'.format(len(train_db)))
train_db , val_db = torch.utils.data.dataset.random_split(train_db,[50000,10000])
print('train length: {},val length: {}'.format(len(train_db),len(val_db)))
train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_db,
    batch_size=batch_size, shuffle=True)

train length: 60000
train length: 50000,val length: 10000


In [10]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784,200),
            nn.ReLU(inplace=True),
            nn.Linear(200,200),
            nn.ReLU(inplace=True),
            nn.Linear(200,10),
            nn.ReLU(inplace=True)
        )
    def forward(self,x):
        return self.model(x)
    

In [12]:
device = torch.device('cuda:0')
model = MLP().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

for epoch in epochs:
    for idx,(train,target) in enumerate(train_loader):
        train = train.view(-1,28*28)
        train,target = train.to(device),target.cuda()
        logits = model.forward(train)
        loss = criterion(logits,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if idx%100 == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)] \t Loss: {:.6f}'
            .format(epoch,idx*len(train),len(train_loader.dataset),100. * idx / len(train_loader),loss.item()))
            
    test_loss = 0
    correct = 0
    for data, target in val_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()
 
        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()
    test_loss /= len(val_loader.dataset)
    print('\nVAL set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(val_loader.dataset),
         100. * correct / len(val_loader.dataset)))

RuntimeError: Error attempting to use dtype torch.float32 with layout torch.strided and device type CUDA.  Torch not compiled with CUDA enabled.
